<a href="https://colab.research.google.com/github/rselent/this_was_a_test/blob/master/build2_Asteroids_predicting_PHAs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# I don't know how replicable this is per user, but for now this should let me
# access my CSV, which is stored on Google Drive instead of github due to size
# (currently >100mb, maybe less after it's cleaned)

# If I perchance get it under 100mb, I'll migrate it over to github, but that
# won't be a primary (nor secondary) objective

from google.colab import drive
drive.mount( '/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [48]:
import pandas as pd

# New dataset, self-generated from JPL Small-Body Database Search Engine, 1/6/2020, 5:10pm
df = pd.read_csv( 'drive/My Drive/Colab Notebooks/Lambda School DS/Unit 2/Build Data/Asteroids_2020.csv', low_memory= False)
# Original dataset, found on Kaggle, generated from same source (JPL) but by a 3rd party, unknown date
df_old = pd.read_csv( 'drive/My Drive/Colab Notebooks/Lambda School DS/Unit 2/Build Data/Asteroid.csv', low_memory= False)

# df_old will be deprecated by week's end to reduce end filesize, but keeping for now (along with appropriate notations) to show process
df_old.shape, df.shape

((839714, 31), (921901, 34))

In [54]:
# I pulled the same datapoints / features from the original set, changed 1 (name -> full_name), and added 3 more:
# pdes - primary designation (more complete name list, presumably/just-in-case),
# and H_sigma and diameter_sigma - 1-sigma uncertainty values
# It seems ~80k more objects had been identified since the Kaggle set was generated and mine :o

df.head(20)

,full_name,pdes,a,e,i,om,w,q,ad,per_y,data_arc,condition_code,n_obs_used,H,H_sigma,neo,pha,diameter,diameter_sigma,extent,albedo,rot_per,GM,BV,UB,IR,spec_B,spec_T,G,moid,class,n,per,ma
0,1 Ceres,1,2.769165,0.076009,10.594067,80.305532,73.597694,2.558684,2.979647,4.608202,8822.0,0,1002,3.34,NaN,N,N,939.4,0.200,964.4 x 964.2 x 891.8,0.0900,9.074170,62.6284,0.713,0.426,NaN,C,G,0.12,1.594780,MBA,0.213885,1683.145708,77.372096
1,2 Pallas,2,2.772466,0.230337,34.836234,173.080063,310.048857,2.133865,3.411067,4.616444,72318.0,0,8490,4.13,NaN,N,N,545,18.000,582x556x500,0.1010,7.813200,14.3000,0.635,0.284,NaN,B,B,0.11,1.233240,MBA,0.213503,1686.155999,59.699133
2,3 Juno,3,2.669150,0.256942,12.988918,169.852756,248.138627,1.983332,3.354967,4.360814,72901.0,0,7109,5.33,NaN,N,N,246.596,10.594,NaN,0.2140,7.210000,NaN,0.824,0.433,NaN,Sk,S,0.32,1.034540,MBA,0.226019,1592.787286,34.925016
3,4 Vesta,4,2.361418,0.088721,7.141771,103.810804,150.728541,2.151909,2.570926,3.628837,24288.0,0,9325,3.20,NaN,N,N,525.4,0.200,572.6 x 557.2 x 446.4,0.4228,5.342128,17.8000,0.782,0.492,NaN,V,V,0.32,1.139480,MBA,0.271609,1325.432765,95.861936
4,5 Astraea,5,2.574249,0.191095,5.366988,141.576605,358.687607,2.082324,3.066174,4.130323,63519.0,0,2926,6.85,NaN,N,N,106.699,3.140,NaN,0.2740,16.806000,NaN,0.826,0.411,NaN,S,S,NaN,1.095890,MBA,0.238632,1508.600458,282.366289
5,6 Hebe,6,2.425160,0.203007,14.737901,138.640203,239.807490,1.932835,2.917485,3.776755,62329.0,0,6034,5.71,NaN,N,N,185.18,2.900,NaN,0.2679,7.274500,NaN,0.822,0.399,NaN,S,S,0.24,0.973965,MBA,0.260972,1379.459705,86.197923
6,7 Iris,7,2.385334,0.231206,5.523651,259.563231,145.265106,1.833831,2.936837,3.684105,62452.0,0,5206,5.51,NaN,N,N,199.83,10.000,NaN,0.2766,7.139000,NaN,0.855,0.484,NaN,S,S,NaN,0.846100,MBA,0.267535,1345.619196,140.419656
7,8 Flora,8,2.201764,0.156499,5.886955,110.889330,285.287462,1.857190,2.546339,3.267115,62655.0,0,2744,6.49,NaN,N,N,147.491,1.025,NaN,0.2260,12.865000,NaN,0.885,0.489,NaN,NaN,S,0.28,0.874176,MBA,0.301681,1193.313717,194.882895
8,9 Metis,9,2.385637,0.123114,5.576816,68.908577,6.417369,2.091931,2.679342,3.684806,61821.0,0,2649,6.28,NaN,N,N,190,NaN,NaN,0.1180,5.079000,NaN,0.858,0.496,NaN,NaN,S,0.17,1.106910,MBA,0.267484,1345.875362,276.861623
9,10 Hygiea,10,3.141539,0.112461,3.831560,283.202165,312.315210,2.788240,3.494839,5.568291,62282.0,0,3442,5.43,NaN,N,N,407.12,6.800,NaN,0.0717,27.630000,7.0000,0.696,0.351,NaN,C,C,NaN,1.778390,MBA,0.177007,2033.818281,152.184851


In [55]:
df.isnull().sum()

full_name              0
pdes                   0
a                      0
e                      0
i                      0
om                     0
w                      0
q                      0
ad                     5
per_y                  1
data_arc           38811
condition_code       901
n_obs_used             0
H                   2323
H_sigma           734362
neo                    5
pha                39715
diameter          784793
diameter_sigma    784921
extent            921883
albedo            785978
rot_per           903105
GM                921887
BV                920880
UB                920922
IR                921900
spec_B            920235
spec_T            920921
G                 921782
moid               39715
class                  0
n                      0
per                    5
ma                     1
dtype: int64

In [0]:
df1 = df.drop( columns= {'full_name', 'pdes', 'data_arc', 'H_sigma', 'diameter',
                         'diameter_sigma', 'extent', 'albedo', 'rot_per',
                         'GM', 'BV', 'UB', 'IR', 'spec_B', 'spec_T', 'G'})

In [0]:
# Sigma features, while showing interesting/more precise data (or just precision
# metrics?), do not seem to be worth pursuing re: my objective. There are too many
# missing values in 'H_sigma', for example. 
# The delta between 'diameter' and 'diameter_sigma' is not all that large, however,
# so that could be an interesting secondary or perhaps tertiary objective.

# That said, right off the bat I will be dropping all features with more than 50%
# nan values, as not only is that just straight prohibitive to making any sort of 
# predictions of value, they also, coincidentally, happen to be the most 
# inconsequential features of the dataset (for my needs, and from my knowledge)

In [65]:
df1.shape

(921901, 18)

In [66]:
df1.head()

,a,e,i,om,w,q,ad,per_y,condition_code,n_obs_used,H,neo,pha,moid,class,n,per,ma
0,2.769165,0.076009,10.594067,80.305532,73.597694,2.558684,2.979647,4.608202,0,1002,3.34,N,N,1.59478,MBA,0.213885,1683.145708,77.372096
1,2.772466,0.230337,34.836234,173.080063,310.048857,2.133865,3.411067,4.616444,0,8490,4.13,N,N,1.23324,MBA,0.213503,1686.155999,59.699133
2,2.669150,0.256942,12.988918,169.852756,248.138627,1.983332,3.354967,4.360814,0,7109,5.33,N,N,1.03454,MBA,0.226019,1592.787286,34.925016
3,2.361418,0.088721,7.141771,103.810804,150.728541,2.151909,2.570926,3.628837,0,9325,3.20,N,N,1.13948,MBA,0.271609,1325.432765,95.861936
4,2.574249,0.191095,5.366988,141.576605,358.687607,2.082324,3.066174,4.130323,0,2926,6.85,N,N,1.09589,MBA,0.238632,1508.600458,282.366289


In [67]:
df1.isnull().sum()

a                     0
e                     0
i                     0
om                    0
w                     0
q                     0
ad                    5
per_y                 1
condition_code      901
n_obs_used            0
H                  2323
neo                   5
pha               39715
moid              39715
class                 0
n                     0
per                   5
ma                    1
dtype: int64

In [0]:
# TODO: Everything?

# TODO2: Do 'pha' and 'moid' share the same Nans? <investigate>

# TODO3: Maybe split those observations, make them an ultimate test/prediction case? Or at least part of? Will that work?

# TODO4: I don't want to impute any of these Nan values; is there an algorithm/model that can accommodate? LightGBM? CatBoost?

# TODO5: <todo>